# Supervisor (Polyglot)

In [ ]:
#!import ../nbs/Testing.dib

In [ ]:
#r @"../../../../../../../.nuget/packages/fsharp.control.asyncseq/3.2.1/lib/netstandard2.1/FSharp.Control.AsyncSeq.dll"
#r @"../../../../../../../.nuget/packages/system.reactive/6.0.1-preview.1/lib/net6.0/System.Reactive.dll"
#r @"../../../../../../../.nuget/packages/system.reactive.linq/6.0.1-preview.1/lib/netstandard2.0/System.Reactive.Linq.dll"
#r @"../../../../../../../.nuget/packages/argu/6.1.1/lib/netstandard2.0/Argu.dll"
#r @"../../../../../../../.nuget/packages/system.commandline/2.0.0-beta4.22272.1/lib/net6.0/System.CommandLine.dll"
#r @"../../../../../../../.nuget/packages/microsoft.aspnetcore.http.connections.common/7.0.0/lib/net7.0/Microsoft.AspNetCore.Http.Connections.Common.dll"
#r @"../../../../../../../.nuget/packages/microsoft.aspnetcore.http.connections.client/7.0.0/lib/net7.0/Microsoft.AspNetCore.Http.Connections.Client.dll"
#r @"../../../../../../../.nuget/packages/microsoft.aspnetcore.signalr.common/7.0.0/lib/net7.0/Microsoft.AspNetCore.SignalR.Common.dll"
#r @"../../../../../../../.nuget/packages/microsoft.aspnetcore.signalr.client/7.0.0/lib/net7.0/Microsoft.AspNetCore.SignalR.Client.dll"
#r @"../../../../../../../.nuget/packages/microsoft.aspnetcore.signalr.client.core/7.0.0/lib/net7.0/Microsoft.AspNetCore.SignalR.Client.Core.dll"
#r @"../../../../../../../.nuget/packages/fsharp.json/0.4.1/lib/netstandard2.0/FSharp.Json.dll"

In [ ]:
#!import ../nbs/Common.fs
#!import ../nbs/CommonFSharp.fs
#!import ../nbs/Async.fs
#!import ../nbs/AsyncSeq.fs
#!import ../nbs/Networking.fs
#!import ../nbs/Runtime.fs
#!import ../nbs/FileSystem.fs

In [ ]:
open Common
open FileSystem.Operators
open Microsoft.AspNetCore.SignalR.Client

## sendJson

In [ ]:
let inline sendJson (port : int) (json : string) = async {
    let! portOpen = Networking.testPortOpen port
    if portOpen then
        try
            let connection = HubConnectionBuilder().WithUrl($"http://127.0.0.1:{port}").Build()
            do! connection.StartAsync () |> Async.AwaitTask
            let! result = connection.InvokeAsync<string>("ClientToServerMsg", json) |> Async.AwaitTask
            do! connection.StopAsync () |> Async.AwaitTask
            trace Debug (fun () -> $"sendJson / port: {port} / json: {json} / result.Length: {result |> Option.ofObj |> Option.map String.length}") getLocals
            return Some result
        with ex ->
            trace Critical (fun () -> $"sendJson / port: {port} / json: {json} / ex: {ex |> printException}") getLocals
            return None
    else
        trace Debug (fun () -> "sendJson / error: port not open") getLocals
        return None
}

## sendObj

In [ ]:
let inline sendObj port obj =
    obj
    |> System.Text.Json.JsonSerializer.Serialize
    |> sendJson port

## awaitCompiler

In [ ]:
type VSCPos = {| line : int; character : int |}
type VSCRange = VSCPos * VSCPos
type RString = VSCRange * string
type TracedError = {| trace : string list; message : string |}
type ClientErrorsRes =
    | FatalError of string
    | TracedError of TracedError
    | PackageErrors of {| uri : string; errors : RString list |}
    | TokenizerErrors of {| uri : string; errors : RString list |}
    | ParserErrors of {| uri : string; errors : RString list |}
    | TypeErrors of {| uri : string; errors : RString list |}

In [ ]:
let inline awaitCompiler port cancellationToken = async {
    let! ct =
        cancellationToken
        |> Option.defaultValue System.Threading.CancellationToken.None
        |> Async.mergeCancellationTokenWithDefaultAsync

    let compiler = MailboxProcessor.Start (fun inbox -> async {
        let! availablePort = Networking.getAvailablePort (Some 60) port
        if port = 13805 && availablePort <> port then
            inbox.Post port
        else
            let repositoryRoot = FileSystem.getSourceDirectory () |> FileSystem.findParent ".paket" false

            let compilerPath =
                repositoryRoot </> "deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release"
                |> System.IO.Path.GetFullPath

            let dllPath = compilerPath </> "Spiral.dll"

            let! exitCode, result =
                Runtime.executeWithOptionsAsync
                    {
                        Command = $@"dotnet ""{dllPath}"" port={availablePort}"
                        CancellationToken = Some ct
                        WorkingDirectory = None
                        OnLine = Some <| fun { Line = line } -> async {
                            if line |> String.contains $"Server bound to: http://localhost:{availablePort}" then
                                do! Networking.waitForPortAccess (Some 10000) true availablePort |> Async.Ignore
                                inbox.Post availablePort
                        }
                    }
            trace Debug (fun () -> $"startSupervisor / exitCode: {exitCode} / result: {result}") getLocals
    }, ct)

    let! serverPort = compiler.Receive ()

    let connection = HubConnectionBuilder().WithUrl($"http://127.0.0.1:{serverPort}").Build ()
    do! connection.StartAsync () |> Async.AwaitTask

    let event = Event<_> ()
    let disposable = connection.On<string> ("ServerToClientMsg", event.Trigger)
    let stream =
        FSharp.Control.AsyncSeq.unfoldAsync
            (fun () -> async {
                let! msg = event.Publish |> Async.AwaitEvent
                return Some (msg |> FSharp.Json.Json.deserialize<ClientErrorsRes>, ())
            })
            ()

    let disposable =
        newDisposable (fun () ->
            disposable.Dispose ()
            connection.StopAsync () |> Async.AwaitTask |> Async.StartImmediate
        )

    return
        serverPort,
        stream,
        disposable
}

## getFileUri

In [ ]:
let inline getFileUri path =
    $"file:///{path |> String.trimStart [| '/' |]}"

In [ ]:
let inline getFilePathFromUri uri =
    match System.Uri.TryCreate (uri, System.UriKind.Absolute) with
    | true, uri -> uri.AbsolutePath |> System.IO.Path.GetFullPath
    | _ -> failwith "invalid uri"

## buildFile

In [ ]:
let inline buildFile timeout cancellationToken path = async {
    let fullPath = path |> System.IO.Path.GetFullPath
    let fileDir = fullPath |> System.IO.Path.GetDirectoryName
    let fileName = fullPath |> System.IO.Path.GetFileNameWithoutExtension
    let! code = fullPath |> FileSystem.readAllTextAsync

    let stream, disposable = FileSystem.watchDirectory true fileDir
    use _ = disposable

    let port =
        if fullPath |> String.startsWith (System.IO.Path.GetTempPath ())
            && fullPath |> String.contains "Microsoft.DotNet.Interactive.App" |> not
        then 13807
        else 13805

    use cts = new System.Threading.CancellationTokenSource ()
    use cts =
        System.Threading.CancellationTokenSource.CreateLinkedTokenSource
            [| cts.Token; cancellationToken |> Option.defaultValue System.Threading.CancellationToken.None |]
    use _ = newDisposable cts.Cancel
    let! serverPort, errors, disposable = awaitCompiler port (Some cts.Token)
    use _ = disposable

    let fsxContentSeq =
        stream
        |> FSharp.Control.AsyncSeq.choose (function
            | _, FileSystem.FileSystemChange.Changed (path, Some content) when path = $"{fileName}.fsx" ->
                Some content
            | _ -> None
        )
        |> FSharp.Control.AsyncSeq.map (fun content ->
            Some (content |> String.replace "\r\n" "\n"), None
        )

    let inline printErrorData (data : {| uri : string; errors : RString list |}) =
        let fileName = data.uri |> System.IO.Path.GetFileName
        let errors =
            data.errors
            |> List.map snd
            |> String.concat "\n"
        $"{fileName}:\n{errors}"

    let errorsSeq =
        errors
        |> FSharp.Control.AsyncSeq.choose (fun error ->
            match error with
            | FatalError message ->
                Some (message, true, error)
            | TracedError data ->
                Some (data.message, true, error)
            | PackageErrors data when data.errors |> List.isEmpty |> not ->
                Some (data |> printErrorData, true, error)
            | TokenizerErrors data when data.errors |> List.isEmpty |> not ->
                Some (data |> printErrorData, true, error)
            | ParserErrors data when data.errors |> List.isEmpty |> not ->
                Some (data |> printErrorData, false, error)
            | TypeErrors data when data.errors |> List.isEmpty |> not ->
                Some (data |> printErrorData, true, error)
            | _ -> None
        )
        |> FSharp.Control.AsyncSeq.map (fun (message, fatal, error) ->
            None, Some (message, fatal, error)
        )

    let outputSeq =
        [fsxContentSeq; errorsSeq]
        |> FSharp.Control.AsyncSeq.mergeAll

    let! outputChild =
        ((None, [], None), outputSeq)
        ||> FSharp.Control.AsyncSeq.scan (
            fun (fsxContentResult, errors, firstErrorTicks) (fsxContent, error) ->
                match fsxContent, error with
                | Some fsxContent, None -> Some fsxContent, errors, firstErrorTicks
                | None, Some (message, fatal, error) ->
                    fsxContentResult,
                    (message, error) :: errors,
                    firstErrorTicks
                    |> Option.defaultWith (fun () ->
                        if fatal
                        then System.DateTime.MinValue.Ticks
                        else System.DateTime.Now.Ticks
                    )
                    |> Some
                | _ -> fsxContentResult, errors, firstErrorTicks
        )
        |> FSharp.Control.AsyncSeq.takeWhileInclusive (fun (fsxContent, errors, firstErrorTicks) ->
            let firstErrorMs =
                firstErrorTicks
                |> Option.map (fun t -> System.TimeSpan(System.DateTime.Now.Ticks - t).TotalMilliseconds)
            trace Debug (fun () -> $"buildFile / fsxContent: {fsxContent} / errors: {errors} / firstErrorMs: {firstErrorMs}") getLocals
            match fsxContent, errors, firstErrorMs with
            | None, [], _ -> true
            | None, _, Some firstErrorMs when firstErrorMs < 2000. -> true
            | _ -> false
        )
        |> FSharp.Control.AsyncSeq.map (fun (fsxContent, errors, _) ->
            fsxContent, errors |> List.distinct
        )
        |> FSharp.Control.AsyncSeq.tryLast
        |> Async.catch
        |> Async.runWithTimeoutAsync timeout
        |> Async.StartChild

    let fileOpenObj = {| FileOpen = {| uri = fullPath |> getFileUri; spiText = code |} |}
    let! _fileOpenResult = fileOpenObj |> sendObj serverPort

    let buildFileObj = {| BuildFile = {| uri = fullPath |> getFileUri; backend = "Fsharp" |} |}
    let! _buildFileResult = buildFileObj |> sendObj serverPort

    return!
        outputChild
        |> Async.map (function
            | Some (Ok (Some result)) -> result
            | Some (Error ex) ->
                trace Critical (fun () -> $"buildFile / error: {ex |> printException}") getLocals
                None, []
            | _ -> None, []
        )
}

## persistCode

In [ ]:
let inline persistCode timeout cancellationToken code = async {
    let tempDir = FileSystem.createTempDirectory ()

    let mainPath = tempDir </> "main.spi"
    do! code |> FileSystem.writeAllTextAsync mainPath

    let repositoryRoot = FileSystem.getSourceDirectory () |> FileSystem.findParent ".paket" false

    let spiprojPath = tempDir </> "package.spiproj"
    let spiprojCode =
        $"""packageDir: {repositoryRoot </> "spiral"}
packages:
    |core-
    fsharp
modules:
    main
"""
    do! spiprojCode |> FileSystem.writeAllTextAsync spiprojPath

    let disposable = newDisposable (fun () ->
        ()
        // tempDir |> FileSystem.deleteDirectoryAsync |> Async.Ignore |> Async.RunSynchronously
    )

    return mainPath, disposable
}

## buildCode

In [ ]:
let inline buildCode timeout cancellationToken code = async {
    let! mainPath, disposable = persistCode timeout cancellationToken code
    use _ = disposable
    return! mainPath |> buildFile timeout cancellationToken
}

In [ ]:
//// test

let buildCode timeout cancellationToken code = buildCode timeout cancellationToken code

In [ ]:
//// test

"""inl app () =
    fsharp.console.write_line "text"
    1i32

inl main () =
    app
    |> dyn
    |> ignore
"""
|> buildCode 8000 None
|> Async.runWithTimeout 8000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        Some """let rec closure0 () () : int32 =
    let v0 : string = "text"
    System.Console.WriteLine v0
    1
let v0 : (unit -> int32) = closure0()
()
""",
        []
    )
)

00:00:00 #1 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230926-1520-3339-3956-330b2aff924b / result: { CreationTime = 2023-09-26 3:20:33 PM
  Exists = true }
00:00:00 #2 [Debug] runWithTimeoutAsync / timeout: 60
00:00:00 #3 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13807"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-930> }
00:00:00 #4 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:00 #5 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:01 #6 [Debug] > Server bound to: http://localhost:13807
00:00:01 #7 [Debug] buildFile / fsxContent:  / errors: [] / firstErrorMs:
00:00:01 #8 [Debug] sendJson / port: 13807 / json: {"FileOpen":{"spiT

In [ ]:
//// test

"inl app () =
    0i32

inl a = 1

inl main () =
    app
    |> dyn
    |> ignore
"
|> buildCode 8000 None
|> Async.runWithTimeout 8000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        Some "let rec closure0 () () : int32 =
    0
let v0 : (unit -> int32) = closure0()
()
",
        ["main.spi:
Global inl/let statements should all return functions known at parse time."]
    )
)

00:00:03 #13 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230926-1520-3668-6849-667b2b4135a7 / result: { CreationTime = 2023-09-26 3:20:36 PM
  Exists = true }
00:00:03 #14 [Debug] runWithTimeoutAsync / timeout: 60
00:00:03 #15 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13807"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-930> }
00:00:03 #16 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:03 #17 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:04 #18 [Debug] > Server bound to: http://localhost:13807
00:00:04 #19 [Debug] buildFile / fsxContent:  / errors: [] / firstErrorMs:
00:00:04 #20 [Debug] sendJson / port: 13807 / json: {"FileOpen

In [ ]:
//// test

""
|> buildCode 4000 None
|> Async.runWithTimeout 4000
|> _equal None

00:00:05 #27 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230926-1520-3923-2352-2bd2fc2dd5d1 / result: { CreationTime = 2023-09-26 3:20:39 PM
  Exists = true }
00:00:05 #28 [Debug] runWithTimeoutAsync / timeout: 60
00:00:05 #29 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13807"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-930> }
00:00:06 #30 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:06 #31 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:06 #32 [Debug] > Server bound to: http://localhost:13807
00:00:06 #33 [Debug] buildFile / fsxContent:  / errors: [] / firstErrorMs:
00:00:06 #34 [Debug] sendJson / port: 13807 / json: {"FileOpen

In [ ]:
//// test

"""inl main () =
    1i32 / 0i32
"""
|> buildCode 8000 None
|> Async.runWithTimeout 8000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        None,
        ["An attempt to divide by zero has been detected at compile time."]
    )
)

00:00:10 #41 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230926-1520-4347-4720-4365eb8b5cf1 / result: { CreationTime = 2023-09-26 3:20:43 PM
  Exists = true }
00:00:10 #42 [Debug] runWithTimeoutAsync / timeout: 60
00:00:10 #43 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13807"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-930> }
00:00:10 #44 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:10 #45 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:10 #47 [Debug] > Server bound to: http://localhost:13807
00:00:10 #48 [Debug] buildFile / fsxContent:  / errors: [] / firstErrorMs:
00:00:11 #49 [Debug] sendJson / port: 13807 / json: {"FileOpen

In [ ]:
//// test

"""union a =
    | B
    | c

inl main () =
    ()
"""
|> buildCode 8000 None
|> Async.runWithTimeout 8000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        Some "()
",
        ["main.spi:
Expected: uppercase variable"]
    )
)

00:00:12 #54 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230926-1520-4571-7176-71350d3c279f / result: { CreationTime = 2023-09-26 3:20:45 PM
  Exists = true }
00:00:12 #55 [Debug] runWithTimeoutAsync / timeout: 60
00:00:12 #56 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13807"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-930> }
00:00:12 #57 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:12 #58 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:13 #59 [Debug] > Server bound to: http://localhost:13807
00:00:13 #60 [Debug] buildFile / fsxContent:  / errors: [] / firstErrorMs:
00:00:13 #61 [Debug] sendJson / port: 13807 / json: {"FileOpen

In [ ]:
//// test

"""
/// abc
inl main () =
    ()
"""
|> buildCode 8000 None
|> Async.runWithTimeout 8000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        None,
        ["main.spi:
Expected: whitespace"]
    )
)

00:00:14 #68 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230926-1520-4812-1218-15b6c78261c1 / result: { CreationTime = 2023-09-26 3:20:48 PM
  Exists = true }
00:00:14 #69 [Debug] runWithTimeoutAsync / timeout: 60
00:00:14 #70 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13807"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-930> }
00:00:15 #71 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:15 #72 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:15 #73 [Debug] > Server bound to: http://localhost:13807
00:00:15 #74 [Debug] buildFile / fsxContent:  / errors: [] / firstErrorMs:
00:00:15 #75 [Debug] sendJson / port: 13807 / json: {"FileOpen

In [ ]:
//// test

"""
inl main () =
    real
        inl real_unbox forall a. (obj : a) : a =
            typecase obj with
            | _ => obj
        real_unbox ()
    ()
"""
|> buildCode 8000 None
|> Async.runWithTimeout 8000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        None,
        ["Cannot apply a forall with a term."]
    )
)

00:00:16 #79 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230926-1520-4976-7681-7bafa5b9aeac / result: { CreationTime = 2023-09-26 3:20:49 PM
  Exists = true }
00:00:16 #80 [Debug] runWithTimeoutAsync / timeout: 60
00:00:16 #81 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13807"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-930> }
00:00:16 #82 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:16 #83 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:17 #84 [Debug] > Server bound to: http://localhost:13807
00:00:17 #85 [Debug] buildFile / fsxContent:  / errors: [] / firstErrorMs:
00:00:17 #86 [Debug] sendJson / port: 13807 / json: {"FileOpen

In [ ]:
//// test

"""
inl main () =
    real
        inl real_unbox forall a. (obj : a) : a =
            typecase obj with
            | _ => obj
        real_unbox `i32 1
"""
|> buildCode 8000 None
|> Async.runWithTimeout 8000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        None,
        ["The main function should not have a forall."]
    )
)

00:00:18 #91 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230926-1520-5209-0945-00cae7e1b1cb / result: { CreationTime = 2023-09-26 3:20:52 PM
  Exists = true }
00:00:18 #92 [Debug] runWithTimeoutAsync / timeout: 60
00:00:18 #93 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13807"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-930> }
00:00:18 #94 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:18 #95 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:19 #96 [Debug] > Server bound to: http://localhost:13807
00:00:19 #97 [Debug] buildFile / fsxContent:  / errors: [] / firstErrorMs:
00:00:19 #98 [Debug] sendJson / port: 13807 / json: {"FileOpen

In [ ]:
//// test

"""
inl init_series start end inc =
    inl total : f64 = conv ((end - start) / inc) + 1
    listm.init total (conv >> (*) inc >> (+) start) : list f64

type integration = (f64 -> f64) -> f64 -> f64 -> f64

inl integral dt : integration =
    fun f a b =>
        init_series (a + dt / 2) (b - dt / 2) dt
        |> listm.map (f >> (*) dt)
        |> listm.fold (+) 0

inl main () =
    integral 0.1 (fun x => x ** 2) 0 1
"""
|> buildCode 8000 None
|> Async.runWithTimeout 8000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        Some "0.3325000000000001\n",
        []
    )
)

00:00:20 #103 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230926-1520-5423-2392-254783fe3854 / result: { CreationTime = 2023-09-26 3:20:54 PM
  Exists = true }
00:00:20 #104 [Debug] runWithTimeoutAsync / timeout: 60
00:00:20 #105 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13807"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-930> }
00:00:21 #106 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:21 #107 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:21 #108 [Debug] > Server bound to: http://localhost:13807
00:00:21 #109 [Debug] buildFile / fsxContent:  / errors: [] / firstErrorMs:
00:00:21 #110 [Debug] sendJson / port: 13807 / json: {"

In [ ]:
//// test

"""
inl init_series start end inc =
    inl total : f64 = conv ((end - start) / inc) + 1
    listm.init total (conv >> (*) inc >> (+) start) : list f64

type integration = (f64 -> f64) -> f64 -> f64 -> f64

inl integral dt : integration =
    fun f a b =>
        init_series (a + dt / 2) (b - dt / 2) dt
        |> listm.map (f >> (*) dt)
        |> listm.fold (+) 0

inl main () =
    integral 0.01 (fun x => x ** 2) 0 1
"""
|> buildCode 8000 None
|> Async.runWithTimeout 8000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
// |> _equal None
|> fun x -> printfn $"{x.ToDisplayString ()}"

00:00:23 #115 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230926-1520-5656-5654-5d33c9ba8a96 / result: { CreationTime = 2023-09-26 3:20:56 PM
  Exists = true }
00:00:23 #116 [Debug] runWithTimeoutAsync / timeout: 60
00:00:23 #117 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13807"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-930> }
00:00:23 #118 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:23 #119 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:24 #120 [Debug] > Server bound to: http://localhost:13807
00:00:24 #121 [Debug] buildFile / fsxContent:  / errors: [] / firstErrorMs:
00:00:24 #122 [Debug] sendJson / port: 13807 / json: {"

## getFileTokenRange

In [ ]:
let inline getFileTokenRange timeout cancellationToken path = async {
    let fullPath = path |> System.IO.Path.GetFullPath
    let! code = fullPath |> FileSystem.readAllTextAsync
    let lines = code |> String.split [| '\n' |]

    let port =
        if fullPath |> String.startsWith (System.IO.Path.GetTempPath ())
            && fullPath |> String.contains "Microsoft.DotNet.Interactive.App" |> not
        then 13807
        else 13805

    let! serverPort, outputs, disposable = awaitCompiler port cancellationToken
    use _ = disposable

    let fileOpenObj = {| FileOpen = {| uri = fullPath |> getFileUri; spiText = code |} |}
    let! fileOpenResult = fileOpenObj |> sendObj serverPort

    let fileTokenRangeObj =
        {|
            FileTokenRange =
                {|
                    uri = fullPath |> getFileUri
                    range =
                        [|
                            {| line = 0; character = 0 |}
                            {| line = lines.Length - 1; character = lines.[lines.Length - 1].Length |}
                        |]
                |}
        |}
    let! fileTokenRangeResult = fileTokenRangeObj |> sendObj serverPort

    return fileTokenRangeResult |> Option.map FSharp.Json.Json.deserialize<int array>
}

## getCodeTokenRange

In [ ]:
let inline getCodeTokenRange timeout cancellationToken code = async {
    let! mainPath, disposable = persistCode timeout cancellationToken code
    use _ = disposable
    return! mainPath |> getFileTokenRange timeout cancellationToken
}

In [ ]:
//// test

"""inl main () = ()"""
|> getCodeTokenRange 8000 None
|> Async.runWithTimeout 8000
|> Option.flatten
|> _equal (Some [|0; 0; 3; 7; 0; 0; 4; 4; 0; 0; 0; 5; 1; 8; 0; 0; 1; 1; 8; 0; 0; 2; 1; 4; 0; 0;
2; 1; 8; 0; 0; 1; 1; 8; 0|])

00:00:39 #535 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230926-1521-1263-6394-683e4a9fc098 / result: { CreationTime = 2023-09-26 3:21:12 PM
  Exists = true }
00:00:39 #536 [Debug] runWithTimeoutAsync / timeout: 60
00:00:39 #537 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13807"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:it@4-372> }
00:00:39 #538 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:39 #539 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:40 #540 [Debug] > Server bound to: http://localhost:13807
00:00:40 #541 [Debug] sendJson / port: 13807 / json: {"FileOpen":{"spiText":"inl main () = ()","uri":"file:///C:\\Users\\i574n\\AppData\\L

In [ ]:
//// test

"""inl main () = 1i32"""
|> getCodeTokenRange 8000 None
|> Async.runWithTimeout 8000
|> Option.flatten
|> _equal (Some [|0; 0; 3; 7; 0; 0; 4; 4; 0; 0; 0; 5; 1; 8; 0; 0; 1; 1; 8; 0; 0; 2; 1; 4; 0; 0;
2; 1; 3; 0; 0; 1; 3; 12; 0|])

00:00:43 #543 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230926-1521-1719-1908-199e62e417ee / result: { CreationTime = 2023-09-26 3:21:17 PM
  Exists = true }
00:00:43 #544 [Debug] runWithTimeoutAsync / timeout: 60
00:00:43 #545 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13807"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:it@4-687> }
00:00:44 #546 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:44 #547 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:44 #548 [Debug] > Server bound to: http://localhost:13807
00:00:44 #549 [Debug] sendJson / port: 13807 / json: {"FileOpen":{"spiText":"inl main () = 1i32","uri":"file:///C:\\Users\\i574n\\AppData\

## Arguments

In [ ]:
[<RequireQualifiedAccess>]
type Arguments =
    | [<Argu.ArguAttributes.Mandatory>] BuildFile of string * string
    | Timeout of int

    interface Argu.IArgParserTemplate with
        member s.Usage =
            match s with
            | BuildFile _ -> nameof Arguments.BuildFile
            | Timeout _ -> nameof Arguments.Timeout

## main

In [ ]:
let main args =
    let argsMap = args |> Runtime.parseArgsMap<Arguments>

    let inputPath, outputPath =
        match argsMap.[nameof Arguments.BuildFile] with
        | [ Arguments.BuildFile (inputPath, outputPath) ] -> Some (inputPath, outputPath)
        | _ -> None
        |> Option.get

    let timeout =
        match argsMap |> Map.tryFind (nameof Arguments.Timeout) with
        | Some [ Arguments.Timeout timeout ] -> timeout
        | _ -> 30000

    async {
        let! outputCode, errors = inputPath |> buildFile timeout None
        
        errors
        |> List.map snd
        |> List.iter (fun error ->
            trace Critical (fun () -> $"main / error: {error}") getLocals
        )

        match outputCode with
        | Some outputCode ->
            do! outputCode |> FileSystem.writeAllTextAsync outputPath
            return 0
        | None ->
            return 1
    }
    |> Async.runWithTimeout timeout
    |> Option.defaultValue 1

In [ ]:
//// test

let args =
    System.Environment.GetEnvironmentVariable "ARGS"
    |> Runtime.splitArgs
    |> Seq.toArray

match args with
| [||] -> 0
| args -> if main args = 0 then 0 else failwith "main failed"

0